In [1]:
######################################## Initialize ##################################

# Basics
from pymongo import MongoClient
import os
import numpy as np
import pandas as pd
import time
import boto3
import io
import warnings
warnings.filterwarnings('ignore')

# import findspark
# findspark.init('/usr/lib/spark')

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()
import time

# Feature Engineering
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer, HashingTF)
from pyspark.sql.functions import length
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import preprocessor as p
from pyspark.sql.functions import dayofyear, concat_ws, collect_list, countDistinct

# Models
from pyspark.ml.classification import LogisticRegression

# Pipeline
from pyspark.ml import Pipeline

# Evaluators
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [2]:
######################################## Data ############################################

#Setup Mongo and create the database and collection
User = os.environ['MONGODB_USER']
password = os.environ['MONGODB_PASS']
IP = os.environ['IP']

client = MongoClient(IP, username=User, password=password)
db = client['stock_tweets']

#Grab references
twitter_coll_reference = db.twitter
iex_coll_reference = db.iex

In [3]:
######################################## Build Twitter Pandas Frame #######################
# Create Data Frame
twitter_data = pd.DataFrame(list(twitter_coll_reference.find()))

# Need to convert the created_at to a time stamp and set to index
twitter_data.index=pd.to_datetime(twitter_data['created_at'])

# Delimited the Company List into separate rows
delimited_twitter_data=[]

for item in twitter_data.itertuples():
    #twitter_dict={}
    for company in item[1]:
        twitter_dict={}
        twitter_dict['created_at']=item[0]
        twitter_dict['company']=company
        twitter_dict['text']=item[11]
        twitter_dict['user_followers_count']=item[12]
        twitter_dict['user_name']=item[13]
        twitter_dict['user_statuses_count']=item[15]
        delimited_twitter_data.append(twitter_dict)

delimited_twitter_df = pd.DataFrame(delimited_twitter_data) 
#delimited_twitter_df.set_index('created_at', inplace=True)

In [4]:
# Convert to Spark Dataframe
# Create a Spark DataFrame from Pandas
twitter_df = spark.createDataFrame(delimited_twitter_df)

In [5]:
twitter_df.show()

+-------+-------------------+--------------------+--------------------+--------------------+-------------------+
|company|         created_at|                text|user_followers_count|           user_name|user_statuses_count|
+-------+-------------------+--------------------+--------------------+--------------------+-------------------+
|   TSLA|2018-03-12 18:07:08|$TSLA so nice so ...|                1703|TradeTherapAnalytics|              67528|
|   AAPL|2018-03-12 18:07:19|@JoKiddo But how ...|                2901|        Gilmo Report|              18524|
|   AAPL|2018-03-12 18:07:23|RT @StockTwits: T...|                5256|           Mark Hill|              13523|
|   GOOG|2018-03-12 18:07:23|RT @StockTwits: T...|                5256|           Mark Hill|              13523|
|  GOOGL|2018-03-12 18:07:23|RT @StockTwits: T...|                5256|           Mark Hill|              13523|
|   AAPL|2018-03-12 18:07:25|$AAPL may be work...|                 486|       William White|    

In [6]:
# Need to Group by Day and company
twitter_daily_df = twitter_df.groupby(dayofyear("created_at"),"Company").count().orderBy('dayofyear(created_at)','Company')
twitter_daily_df = twitter_daily_df.select(col("dayofyear(created_at)").alias("Day"), 
                                           col("Company").alias ("Company"), 
                                           col("count").alias("Number_of_tweets"))

In [7]:
# Combine the Text
combined_text = twitter_df.groupby(dayofyear("created_at"),"Company").agg(concat_ws(" ", collect_list("text"))).orderBy('dayofyear(created_at)','Company')
combined_text = combined_text.select(col("dayofyear(created_at)").alias("Day"), 
                                           col("Company").alias ("Company"), 
                                           col("concat_ws( , collect_list(text))").alias("Text"))


In [8]:
# Add Text Data
twitter_daily_df = twitter_daily_df.join(combined_text, ["Day","Company"]).orderBy('Day','Company')

In [9]:
# Distinct Users
distinct_users = twitter_df.groupby(dayofyear("created_at"),"Company").agg(countDistinct("user_name")).orderBy('dayofyear(created_at)','Company')
distinct_users = distinct_users.select(col("dayofyear(created_at)").alias("Day"), 
                                           col("Company").alias ("Company"), 
                                           col("count(DISTINCT user_name)").alias("Distinct_Users"))



In [10]:
# Add Distinct Users
twitter_daily_df = twitter_daily_df.join(distinct_users, ["Day","Company"]).orderBy('Day','Company')

In [11]:
twitter_daily_df.show()

+---+-------+----------------+--------------------+--------------+
|Day|Company|Number_of_tweets|                Text|Distinct_Users|
+---+-------+----------------+--------------------+--------------+
| 71|   AAPL|             403|@JoKiddo But how ...|           258|
| 71|   AMZN|             275|Amazon hits $1600...|           162|
| 71|     BA|             137|Thus, its cheaper...|            94|
| 71|   BABA|              50|Thus, its cheaper...|            37|
| 71|    BAC|              51|Open an account w...|            35|
| 71|      C|              73|We take a look at...|            59|
| 71|  CMCSA|               8|3/12 Unusual Opti...|             6|
| 71|   CSCO|               6|RT @CabotAnalysts...|             6|
| 71|     FB|             128|RT @stockstreamtv...|            74|
| 71|     GE|              51|.
The signals Mar...|            43|
| 71|   GOOG|             148|RT @StockTwits: T...|           104|
| 71|  GOOGL|              81|RT @StockTwits: T...|           

In [12]:
####################### Build Stock Data ###########################
stock_data = pd.DataFrame(list(iex_coll_reference.find()))

# Need to convert the created_at to a time stamp
stock_data.index=pd.to_datetime(stock_data['latestUpdate'])
stock_data['latestUpdate'] = pd.to_datetime(stock_data['latestUpdate'])
#Group By hourly and stock price
# Need to get the first stock price in teh hour, and then the last to take the difference to see how much change.
stock_delimited_daily = stock_data.sort_values('latestUpdate').groupby([pd.Grouper(freq="D"), 'Ticker']).first()['latestPrice'].to_frame()
stock_delimited_daily.columns = ['First_Price']
stock_delimited_daily['Last_Price'] = stock_data.sort_values('latestUpdate').groupby([pd.Grouper(freq="D"), 'Ticker']).last()['latestPrice']

# Then need to take the difference and turn into a percentage.
stock_delimited_daily['Price_Percent_Change'] = ((stock_delimited_daily['Last_Price'] 
                                                   - stock_delimited_daily['First_Price'])/stock_delimited_daily['First_Price'])*100

# Need to also show Percent from open price
stock_delimited_daily['Open_Price'] = stock_data.groupby([pd.Grouper(freq="D"), 'Ticker'])['open'].mean()
stock_delimited_daily['Price_Percent_Open'] = ((stock_delimited_daily['Last_Price'] 
                                                 - stock_delimited_daily['Open_Price'])/stock_delimited_daily['Open_Price'])*100

# Also include mean volume
stock_delimited_daily['Mean_Volume'] = stock_data.groupby([pd.Grouper(freq="D"), 'Ticker'])['latestVolume'].mean()

# Classification Labels
stock_delimited_daily['Price_Change'] = np.where(stock_delimited_daily['Price_Percent_Change']>=0, 1, 0)
stock_delimited_daily['Open_Price_Change'] = np.where(stock_delimited_daily['Price_Percent_Open']>=0, 1, 0)

# Rename the Index
stock_delimited_daily = stock_delimited_daily.reindex(stock_delimited_daily.index.rename(['Time', 'Company']))

# Flatten Dataframe
stock_delimited_daily.reset_index(inplace=True)


In [13]:
# Create a Spark DataFrame from Pandas
stock_df = spark.createDataFrame(stock_delimited_daily)

In [14]:
stock_df.show()

+-------------------+-------+-----------+----------+--------------------+------------------+--------------------+--------------------+------------+-----------------+
|               Time|Company|First_Price|Last_Price|Price_Percent_Change|        Open_Price|  Price_Percent_Open|         Mean_Volume|Price_Change|Open_Price_Change|
+-------------------+-------+-----------+----------+--------------------+------------------+--------------------+--------------------+------------+-----------------+
|2018-02-26 00:00:00|   PCLN|    1905.64|   1905.64|                 0.0|1905.9499999994837|-0.01626485476972973|            566730.0|           1|                0|
|2018-03-12 00:00:00|   AAPL|     181.73|    181.75|0.011005337588736166|180.22999999999962|  0.8433668090775026|2.7673734089171976E7|           1|                1|
|2018-03-12 00:00:00|   AMZN|   1600.745|   1598.39| -0.1471189977166751| 1592.600000000004|  0.3635564485744119|   4376276.694267516|           0|                1|
|201

In [17]:
stock_df = stock_df.withColumn("Day", dayofyear("Time"))

In [19]:
stock_df.select(["Time", "Day"]).show()

+-------------------+---+
|               Time|Day|
+-------------------+---+
|2018-02-26 00:00:00| 57|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
|2018-03-12 00:00:00| 71|
+-------------------+---+
only showing top 20 rows



In [20]:
#Combine The Data
daily_df = twitter_daily_df.join(stock_df, ["Day","Company"]).orderBy('Day','Company')

In [21]:
daily_df.show(5)

+---+-------+----------------+--------------------+--------------+-------------------+-----------+----------+--------------------+------------------+------------------+--------------------+------------+-----------------+
|Day|Company|Number_of_tweets|                Text|Distinct_Users|               Time|First_Price|Last_Price|Price_Percent_Change|        Open_Price|Price_Percent_Open|         Mean_Volume|Price_Change|Open_Price_Change|
+---+-------+----------------+--------------------+--------------+-------------------+-----------+----------+--------------------+------------------+------------------+--------------------+------------+-----------------+
| 71|   AAPL|             403|@JoKiddo But how ...|           258|2018-03-12 00:00:00|     181.73|    181.75|0.011005337588736166|180.22999999999962|0.8433668090775026|2.7673734089171976E7|           1|                1|
| 71|   AMZN|             275|Amazon hits $1600...|           162|2018-03-12 00:00:00|   1600.745|   1598.39| -0.147

In [23]:
daily_df.printSchema()

root
 |-- Day: integer (nullable = true)
 |-- Company: string (nullable = true)
 |-- Number_of_tweets: long (nullable = false)
 |-- Text: string (nullable = false)
 |-- Distinct_Users: long (nullable = false)
 |-- Time: timestamp (nullable = true)
 |-- First_Price: double (nullable = true)
 |-- Last_Price: double (nullable = true)
 |-- Price_Percent_Change: double (nullable = true)
 |-- Open_Price: double (nullable = true)
 |-- Price_Percent_Open: double (nullable = true)
 |-- Mean_Volume: double (nullable = true)
 |-- Price_Change: long (nullable = true)
 |-- Open_Price_Change: long (nullable = true)

